In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from xgboost.sklearn import XGBRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, train_test_split
import statsmodels.formula.api as smf
from sklearn.ensemble import IsolationForest, RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.decomposition import PCA
import random
import corex
import mca
from sklearn.linear_model import HuberRegressor, BayesianRidge, Lasso
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [ ]:
# Intialising Lists of features
bin_features = None 
cat_features = None
paired_bin_features = None
paired_cat_features = None
trans_cat_features = None
trans_paired_bin_features = None
trans_paired_cat_features = None
mca_features = None
pca_all_features = None

In [3]:
# Coding categerical features with the values of the target variable
from math import exp

def transform_categerical_feature(train, f, target_mean):
    _lambda = lambda n: 1.0 / (1.0 + exp(-(n - 5.0) / 1))
    
    trans_df = train.groupby(f, as_index=False)['y'].agg({'var_mean': 'mean',
                                                       'cnt': 'count'})
    trans_df.loc[:, 'trans_' + f] = trans_df.apply(lambda r: (_lambda(r['cnt']) * r['var_mean']) +
                                                              ((1 - _lambda(r['cnt'])) * target_mean), axis=1)
    
    target_mean = np.mean(train.y.values)
    target_std = np.std(train.y.values)
    
    trans_df = trans_df[[f, 'trans_' + f]]
    
    trans_df.loc[-1] = [-1, target_mean]
    trans_df.index = trans_df.index + 1
    
    trans_df.loc[-1] = [-2, target_std]
    trans_df.index = trans_df.index + 1
    
    return trans_df


def target_coding_column(data, trans_df, field):
    data = data.merge(trans_df, on=field, how='left')    
   
    target_mean = trans_df.loc[(trans_df[field] == -1), 'trans_' + field].values[0]
    target_std = trans_df.loc[trans_df[field] == -2, 'trans_' + field].values[0]
    
    data.loc[np.isnan(data['trans_' + field]), 'trans_' + field] = \
                                        np.random.normal(loc=target_mean, scale=target_std
                                        , size=(len(data.loc[np.isnan(data['trans_' + field])])))

    return data


def target_coding_training(train_df, field):
    kfold = KFold(n_splits=5)
    tmp = pd.DataFrame()

    for itrain, itest in kfold.split(np.zeros(len(train_df))):
        target_mean = np.mean(train_df.iloc[itrain].y.values)

        trans_df = transform_categerical_feature(train_df.iloc[itrain], field, target_mean)
        tmp = tmp.append(target_coding_column(train_df.iloc[itest], trans_df, field))

    target_mean = np.mean(train_df.y.values)
    trans_df = transform_categerical_feature(train_df, field, target_mean)
    return tmp, trans_df

In [4]:
data = pd.read_csv('./train.csv.zip', compression='zip')
test_data = pd.read_csv('./test.csv.zip', compression='zip')

In [5]:
all_features = list(data.columns)        
all_features.remove('y')
all_features.remove('ID')

cat_features = ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'] # Categerical features
bin_features = [ x for x in all_features if x not in cat_features] # Binary featuers

In [6]:
all_data = pd.concat([data, test_data])

# Coding categerical featuers
for c in cat_features:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values))
    data.loc[:, c] = lbl.transform(list(data[c].values))
    test_data.loc[:, c] = lbl.transform(list(test_data[c].values))

In [7]:
test_data['y'] = -1
all_data = pd.concat([data, test_data])

In [8]:
# Identical features that will be disregarded

identical_features = ['X214', 'X227', 'X360', 'X172', 'X364', 'X253', 'X84', 'X213',
                      'X382', 'X216', 'X293', 'X119', 'X302', 'X279', 'X296', 'X134', 
                      'X324', 'X326', 'X39', 'X37', 'X35', 'X76', 'X199', 'X299', 'X113', 
                      'X222', 'X226', 'X102', 'X385', 'X254', 'X147', 'X146', 'X330', 
                      'X262', 'X239', 'X244']

In [9]:
# remove the indentical features
identical_features = list(set(identical_features))
bin_features = [x for x in bin_features if x not in identical_features]
all_features = [x for x in all_features if x not in identical_features]

In [10]:
# Pairing binary features that are more than 99.8% different 
first = True
n = float(len(all_data))
paired_bin_features = []
bin_f = bin_features
for f1 in bin_features[:-1]:
    first = True
    bin_f = bin_f[1:]
    for f2 in bin_f:
        diff = len(all_data.loc[all_data[f1] != all_data[f2]]) / n
        if diff > 0.998:
            if first:
                print
                print 'Paired %s with: %s'%  (f1, f2),
                first = False
            else:
                print f2,
            ff1 = 'X'+str(max([int(f1[1:]), int(f2[1:])]))
            ff2 = 'X'+str(min([int(f1[1:]), int(f2[1:])]))
            paired_bin_features.append(ff1 + '_' + ff2)
            data[ff1+ '_' + ff2] = data[ff1].astype('str') + data[ff2].astype('str')
            test_data[ff1+ '_' + ff2] = test_data[ff1].astype('str') + test_data[ff2].astype('str')


Paired X11 with: X74 X205
Paired X15 with: X74 X205
Paired X29 with: X136 X263
Paired X33 with: X74 X205
Paired X40 with: X205
Paired X42 with: X205
Paired X52 with: X120
Paired X54 with: X136 X263
Paired X59 with: X205
Paired X74 with: X93 X95 X107 X124 X190 X204 X210 X233 X235 X236 X252 X257 X259 X260 X268 X270 X278 X280 X288 X289 X290 X295 X297 X319 X339 X347 X369 X372
Paired X83 with: X205
Paired X86 with: X205
Paired X87 with: X205
Paired X89 with: X205
Paired X93 with: X205
Paired X95 with: X205
Paired X105 with: X205
Paired X107 with: X205
Paired X110 with: X205
Paired X124 with: X205
Paired X128 with: X130
Paired X136 with: X232
Paired X142 with: X158
Paired X145 with: X205
Paired X153 with: X205
Paired X156 with: X157
Paired X160 with: X205
Paired X167 with: X205
Paired X184 with: X205
Paired X186 with: X194
Paired X190 with: X205
Paired X204 with: X205
Paired X205 with: X207 X210 X233 X235 X236 X245 X252 X257 X258 X259 X260 X266 X268 X269 X270 X277 X278 X280 X288 X289 X290 X

In [11]:
len(paired_bin_features)

99

In [12]:
# Pairing all combinations of categerical features
paired_cat_features = []
cat_f = cat_features
for f1 in cat_features[:-1]:
    cat_f = cat_f[1:]
    for f2 in cat_f:
        data[f1+'_'+f2] = data[f1].astype('str') + data[f2].astype('str')
        test_data[f1+ '_' + f2] = test_data[f1].astype('str') + test_data[f2].astype('str')
        paired_cat_features.append(f1+ '_' + f2)
print len(paired_cat_features)

28


In [13]:
# Code categerical feature using one-hot encoder
from sklearn.preprocessing import OneHotEncoder
OHE = OneHotEncoder(sparse=False,dtype=np.int8)
ohe_results = OHE.fit_transform(all_data[cat_features].values)

cat_bin_features = []

results = ohe_results[0:len(data), :]

print 'OHE:', results.shape[1]

for i in range(results.shape[1]):
    data['ohe_' + str(i)] = results[:, i]
    cat_bin_features.append('ohe_' + str(i))


results = ohe_results[len(data):, :]
for i in range(results.shape[1]):
    test_data['ohe_' + str(i)] = results[:, i]
    

all_data = pd.concat([data, test_data])

OHE: 211


In [14]:
# Calculate t-distributed Stochastic Neighbor Embedding for all the binary features
from sklearn.manifold import TSNE
tsne_feat = 5
tsne = TSNE(n_components=tsne_feat, learning_rate=500, verbose=1)
tsne_results = tsne.fit_transform(all_data[bin_features + cat_bin_features].values)

tsne_features = []

results = tsne_results[0:len(data), :]
for i in range(tsne_feat):
    data['tsne_' + str(i)] = results[:, i]
    tsne_features.append('tsne_' + str(i))

results = tsne_results[len(data):, :]
for i in range(tsne_feat):
    test_data['tsne_' + str(i)] = results[:, i]

all_data = pd.concat([data, test_data])

# Use to k-means to clsuter the embedding vectors to create a new categerical feature
from sklearn.cluster import KMeans
c = KMeans(n_clusters=100)
c = c.fit(all_data[tsne_features].values)

data['tsne_cls'] = c.predict(data[tsne_features].values)
test_data['tsne_cls'] = c.predict(test_data[tsne_features].values)
cat_features.append('tsne_cls')

[t-SNE] Computing pairwise distances...
[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Computed conditional probabilities for sample 1000 / 8418
[t-SNE] Computed conditional probabilities for sample 2000 / 8418
[t-SNE] Computed conditional probabilities for sample 3000 / 8418
[t-SNE] Computed conditional probabilities for sample 4000 / 8418
[t-SNE] Computed conditional probabilities for sample 5000 / 8418
[t-SNE] Computed conditional probabilities for sample 6000 / 8418
[t-SNE] Computed conditional probabilities for sample 7000 / 8418
[t-SNE] Computed conditional probabilities for sample 8000 / 8418
[t-SNE] Computed conditional probabilities for sample 8418 / 8418
[t-SNE] Mean sigma: 1.249424
[t-SNE] KL divergence after 100 iterations with early exaggeration: 1.511334
[t-SNE] Error after 225 iterations: 1.511334


In [15]:
# Use Isometric Mapping to calculate embedingof all the binary variables
from sklearn.manifold import Isomap
isomap_feat = 5
isomap = Isomap(n_neighbors=5, n_components=isomap_feat)
results = isomap.fit_transform(data[bin_features + cat_bin_features].values)

isomap_features = []

for i in range(isomap_feat):
    data['isomap_' + str(i)] = results[:, i]
    isomap_features.append('isomap_' + str(i))

results = isomap.transform(test_data[bin_features + cat_bin_features].values)
for i in range(isomap_feat):
    test_data['isomap_' + str(i)] = results[:, i]


all_data = pd.concat([data, test_data])

# Use to k-means to clsuter the embedding vectors to create a new categerical feature
from sklearn.cluster import KMeans
c = KMeans(n_clusters=100)
c = c.fit(all_data[isomap_features].values)

data['isomap_cls'] = c.predict(data[isomap_features].values)
test_data['isomap_cls'] = c.predict(test_data[isomap_features].values)
cat_features.append('isomap_cls')

In [16]:
# Transform all the categerical and paired features by encoding them using the target variable

trans_features = []

for c in cat_features + paired_bin_features + paired_cat_features:
    data, transform_df = target_coding_training(data, c)
    test_data = target_coding_column(test_data, transform_df, c)
    trans_features.append('trans_' + c)
        
data.index = range(len(data))
test_data.index = range(len(test_data))

trans_cat_features = ['trans_'+x for x in cat_features]    
trans_paired_bin_features = ['trans_'+x for x in paired_bin_features]
trans_paired_cat_features = ['trans_'+x for x in paired_cat_features]

In [17]:
test_data['y'] = -1
all_data = pd.concat([data, test_data])

In [18]:
# Calculate Linear dimensionality reduction using Singular Value Decomposition f
# or binary and transformed features

n_feat = 12
print 'PCA ...'

pca = PCA(n_components=n_feat)
results = pca.fit_transform(data[bin_features + trans_features].values)

pca_all_features = []

for i in range(n_feat):
    data['pca_' + str(i)] = results[:, i]
    pca_all_features.append('pca_' + str(i))


results = pca.transform(test_data[bin_features + trans_features].values)
for i in range(n_feat):
    test_data['pca_' + str(i)] = results[:, i]

PCA ...


In [19]:
# Calculate Linear dimensionality reduction using Singular Value Decomposition f
# or all the binary features

print 'PCA bin...'

pca = PCA(n_components=n_feat)
results = pca.fit_transform(data[bin_features + cat_bin_features].values)

pca_bin_features = []

for i in range(n_feat):
    data['pca_bin_' + str(i)] = results[:, i]
    pca_bin_features.append('pca_bin_' + str(i))


results = pca.transform(test_data[bin_features + cat_bin_features].values)
for i in range(n_feat):
    test_data['pca_bin_' + str(i)] = results[:, i]

PCA bin...


In [20]:
# Use Gaussian Random Projection to reduce the dimensionality
print 'GRP...'
from sklearn import random_projection

grp_model = random_projection.GaussianRandomProjection(n_components=n_feat)

results = grp_model.fit_transform(data[bin_features + trans_features].values)

grp_features = []

for i in range(n_feat):
    data['grp_' + str(i)] = results[:, i]
    grp_features.append('grp_' + str(i))

results = grp_model.transform(test_data[bin_features + trans_features].values)
for i in range(n_feat):
    test_data['grp_' + str(i)] = results[:, i]

GRP...


In [21]:
# Multiple Correspondence Analysis to capture any interaction between all categerical and binary features

print 'MCA...'
mca_model = mca.mca(all_data[all_features], cols=all_features)
mca_results = mca_model.fs_r(N=n_feat)

mca_features = []

results = mca_results[0:len(data), :]

for i in range(n_feat):
    data['mca_' + str(i)] = results[:, i]
    mca_features.append('mca_' + str(i))


results = mca_results[len(data):, :]
for i in range(n_feat):
    test_data['mca_' + str(i)] = results[:, i]


MCA...


In [22]:
# Multiple Correspondence Analysis to capture any interaction between all the binary features
print 'MCA bin...'
mca_model1 = mca.mca(all_data[bin_features + cat_bin_features], cols=bin_features + cat_bin_features)
mca_results = mca_model1.fs_r(N=n_feat)

mca_bin_features = []

results = mca_results[0:len(data), :]

for i in range(n_feat):
    data['mca_bin_' + str(i)] = results[:, i]
    mca_bin_features.append('mca_bin_' + str(i))

results = mca_results[len(data):, :]
for i in range(n_feat):
    test_data['mca_bin_' + str(i)] = results[:, i]

MCA bin...


In [23]:
mca_results.shape

(8418, 12)

In [24]:
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,mca_bin_2,mca_bin_3,mca_bin_4,mca_bin_5,mca_bin_6,mca_bin_7,mca_bin_8,mca_bin_9,mca_bin_10,mca_bin_11
0,0,130.81,37,23,20,0,3,27,9,14,...,0.007585,-0.006389,0.007831,0.009172,0.064545,0.023398,-0.046245,-0.021299,0.115203,0.102771
1,6,88.53,37,21,22,4,3,31,11,14,...,-0.006534,-0.013715,0.012769,0.007381,0.008860,0.006770,-0.021232,0.001643,0.014699,0.026331
2,7,76.26,24,24,38,2,3,30,9,23,...,0.086259,0.027286,-0.006102,0.010600,-0.004477,-0.019870,0.002363,0.009309,0.009331,-0.003302
3,9,80.62,24,21,38,5,3,30,11,4,...,0.096478,0.009374,-0.020390,-0.004860,0.005778,-0.004164,-0.010170,-0.008580,-0.020058,-0.001327
4,13,78.02,24,23,38,5,3,14,3,13,...,0.095224,0.008963,-0.017589,-0.009284,0.000874,-0.001328,-0.005175,-0.012752,-0.026793,-0.000678


In [25]:
save_features_list()

test_data.to_csv('./test_prepared.csv', index=False)
data.to_csv('./data_prepared.csv', index=False)

In [26]:
# Stacked machine learning model that uses different algorithms and 
# different set of features for each algorithem

from itertools import product

class stacked_model:
    def _create_models(self, name_list):        
        self.models = {}
        
        for n in name_list:
            if 'xgb' in n:
                self.models[n] = XGBRegressor(reg_alpha=0.5, reg_lambda=4, gamma=0.5,
                                 learning_rate=0.005, n_estimators=1000, subsample=0.95, colsample_bytree=0.5, 
                                 objective='reg:linear', max_depth=4)
            elif 'RF' in n:
                self.models[n] = RandomForestRegressor(n_estimators=1000, max_depth=8, max_features='sqrt')
            elif 'BR' in n:
                self.models[n] = BayesianRidge(alpha_1=0.5)
            elif 'gbm' in n:
                self.models[n] = GradientBoostingRegressor(n_estimators=1000, max_features=0.95,
                                                           learning_rate=0.005, max_depth=4)
            elif 'lASSO' in n:
                self.models[n] = Lasso(alpha=5)

   
    def __init__(self, model_features):
        self.model_names = ['xgb_manifold', 'xgb_grp_bin', 'xgb_pca', 'xgb_mca', 'xgb_bin', 'xgb_trans', 'xgb_pca_bin',
                            'xgb_mca_bin', 'xgb_all', 'xgb_grp_trans',
                            
                            'gbm_manifold', 'gbm_pca', 'gbm_mca', 'gbm_bin', 'gbm_trans', 'gbm_mca_bin', 
                            'gbm_all', 'gbm_grp_bin', 'gbm_grp_trans'
                           
                            'BR_manifold', 'BR_bin', 'BR_trans', 'BR_pca_bin', 'BR_mca_bin',
                           
                            'LASSO_bin', 'LASSO_trans', 'LASSO_pca_bin', 'LASSO_mca_bin', 'LASSO_all', 
                            'lASSO_grp_bin',
                             
                            'RF_grp_bin', 'RF_pca', 'RF_mca', 'RF_trans', 'RF_grp_trans',
                           
                            'xgb_OR_bin', 'xgb_OR_trans', 'xgb_OR_mca_bin', 'gbm_OR_manifold', 'gbm_OR_pca', 
                            'gbm_OR_bin', 'gbm_OR_trans', 'gbm_OR_mca_bin', 'BR_OR_bin', 'BR_OR_trans', 
                            'BR_OR_mca_bin', 'LASSO_OR_bin', 'LASSO_OR_pca_bin', 'RF_OR_grp_bin', 'RF_OR_trans']
        
        self.features = {}
        for n in self.model_names:
            for f in model_features:
                if f in n:
                    self.features[n] = model_features[f]
                    
        self.models = None
        self._create_models(self.model_names)
        

    def fit_one_fold(self, data):
        self._create_models(self.model_names)
        
        for model_name in self.models.keys():
            print model_name,
            if 'OR' in model_name:
                remove_outliers = IsolationForest(contamination=0.05)
                remove_outliers = remove_outliers.fit(data[self.features[model_name]].values)
                outliers = remove_outliers.predict(data[self.features[model_name]].values)
                in_data = data.loc[outliers==1]
            else:
                in_data = data
                
            self.models[model_name] = self.models[model_name].fit(in_data[self.features[model_name]].values, 
                                                                  in_data.y.values)
            
            
    def predict(self, data):
        for model_name in self.models.keys():
            data[model_name] = self.models[model_name].predict(data[self.features[model_name]].values)
        return data
    
    
    def fit_predict(self, data, kfold):
        tmp = pd.DataFrame()
        i = 0
        for itrain, itest in kfold.values():
            print 'Training fold %d: ' % i,
            i += 1
            self.fit_one_fold(data.iloc[itrain])
            x = self.predict(data.iloc[itest])
            tmp = tmp.append(x)
            print 
            
        self.fit_one_fold(data)
        print '\n--------------------------------------------------'
        return tmp
    
    def fit_predict_data(self, data, kfold):
        tmp = pd.DataFrame()
        i = 0
        for itrain, itest in kfold.values():
            print 'Training fold %d: ' % i,
            i += 1
            self.fit_one_fold(data.iloc[itrain])
            x = self.predict(data.iloc[itest])
            tmp = tmp.append(x)
            print 
            
        print '\n--------------------------------------------------'
        return tmp

In [27]:
# Combinig model that take the output of the class above and combine it 
# using two algorithms Random Forest regressor and Bayesian Ridge regression

class combine_model:
    def __init__(self,comb_features, model_features):
        self.stacked_model = None
        self.features = model_features
        self.comb_features = comb_features
        
        self.model_RF = RandomForestRegressor(n_estimators=300, max_depth=8, min_samples_leaf=20, oob_score=True,
                                  min_samples_split=20, max_features=0.25)
    
        self.model_BR = BayesianRidge(alpha_1=0.5, alpha_2=0.001, lambda_1=0.001, lambda_2=0.5, normalize=True)
        
        
    def predict(self, data):
        data = self.stacked_model.predict(data)
        data['y'] = 0.9 * self.model_RF.predict(data[self.comb_features +
                                                    self.stacked_model.models.keys()].values) + \
                         0.1 * self.model_BR.predict(data[self.comb_features + 
                                                       self.stacked_model.models.keys()].values)
       
        return data
    
    def fit(self, data):
        skf = KFold(n_splits=5, shuffle=True, random_state=2016)
        i = 0
        kfold = {}
        for itrain, itest in skf.split(np.zeros(len(data))):
            kfold[i] = (itrain, itest)
            i += 1
            
        self.stacked_model = stacked_model(self.features)
        stack_output = self.stacked_model.fit_predict(data, kfold)
        
        self.model_RF = self.model_RF.fit(stack_output[self.comb_features + self.stacked_model.models.keys()].values,
                                          stack_output.y.values)
        self.model_BR = self.model_BR.fit(stack_output[self.comb_features + self.stacked_model.models.keys()].values,
                                   stack_output.y.values)
        return self
    
    def prepare_data(self, data):
        skf = KFold(n_splits=5, shuffle=True, random_state=7609)
        i = 0
        kfold = {}
        for itrain, itest in skf.split(np.zeros(len(data))):
            kfold[i] = (itrain, itest)
            i += 1
            
        self.stacked_model = stacked_model(self.features)
        stack_output = self.stacked_model.fit_predict_data(data, kfold)
        
        return stack_output
        

In [28]:
# Combining features to create multiple sets of features to be use as 
# an input to different machine learning models

kfold = KFold(n_splits=5, shuffle=True, random_state=9856)
Y_pred = []
Y_true = []
scores = []

model_features = {}
model_features['all'] = bin_features + cat_bin_features  + tsne_features + isomap_features + \
                        trans_features  + grp_features + mca_features + pca_all_features + pca_bin_features + \
                        mca_bin_features
        
model_features['manifold'] = bin_features + cat_bin_features  + tsne_features + isomap_features
model_features['grp_trans'] = trans_features  + grp_features
model_features['grp_bin'] = bin_features + cat_bin_features  + grp_features
model_features['pca'] = trans_features  + pca_all_features
model_features['mca'] = trans_features  + mca_features
model_features['bin'] = bin_features + cat_bin_features
model_features['trans'] = trans_features
model_features['pca_bin'] = bin_features + cat_bin_features + pca_bin_features
model_features['mca_bin'] = bin_features + cat_bin_features + mca_bin_features

In [29]:
# Train the staked model and make prediction

comb_features = bin_features + trans_features
    
model = combine_model(comb_features, model_features)

model = model.fit(data)

test_data = model.predict(test_data)

test_data[['ID', 'y']].to_csv('submission.csv', index=False)

Training fold 0:  xgb_pca_bin gbm_manifold lASSO_grp_bin xgb_mca gbm_trans gbm_OR_trans gbm_grp_transBR_manifold gbm_all RF_grp_bin gbm_OR_manifold BR_bin xgb_mca_bin gbm_OR_bin xgb_OR_trans gbm_mca RF_trans gbm_bin xgb_grp_bin BR_mca_bin xgb_bin BR_trans gbm_mca_bin BR_OR_bin xgb_OR_bin RF_OR_grp_bin RF_mca xgb_trans RF_grp_trans RF_OR_trans xgb_pca BR_OR_trans gbm_pca gbm_OR_pca gbm_grp_bin xgb_manifold xgb_all xgb_OR_mca_bin BR_OR_mca_bin gbm_OR_mca_bin xgb_grp_trans RF_pca BR_pca_bin
Training fold 1:  xgb_pca_bin gbm_manifold lASSO_grp_bin xgb_mca gbm_trans gbm_OR_trans gbm_grp_transBR_manifold gbm_all RF_grp_bin gbm_OR_manifold BR_bin xgb_mca_bin gbm_OR_bin xgb_OR_trans gbm_mca RF_trans gbm_bin xgb_grp_bin BR_mca_bin xgb_bin BR_trans gbm_mca_bin BR_OR_bin xgb_OR_bin RF_OR_grp_bin RF_mca xgb_trans RF_grp_trans RF_OR_trans xgb_pca BR_OR_trans gbm_pca gbm_OR_pca gbm_grp_bin xgb_manifold xgb_all xgb_OR_mca_bin BR_OR_mca_bin gbm_OR_mca_bin xgb_grp_trans RF_pca BR_pca_bin
Training fold 